<a href="https://colab.research.google.com/github/m10k1/ml-learn/blob/main/PaliGemma%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%A6%E3%81%BF%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')


必要なライブラリを読み込みます。
* torch
* immutabledict
* sentencepiece



In [3]:
!pip install -q -U torch immutabledict sentencepiece

パラメータをダウンロードします。

In [10]:
# variantとマシンタイプを選択
VARIANT = '2b-it'
MACHINE_TYPE = 'cuda'

CONFIG = VARIANT[:2]
if CONFIG == '2b':
  CONFIG = '2b-v2'

import os
import kagglehub

# パラメータの読み込み
weights_dir = kagglehub.model_download(f'google/gemma-2/pyTorch/gemma-2-{VARIANT}')


# tokenizerがあるか確認
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

# checkpointが存在するか確認
ckpt_path = os.path.join(weights_dir, f'model.ckpt')
assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'


# モデルの実装をダウンロード

In [6]:
# NOTE: The "installation" is just cloning the repo.
!git clone https://github.com/google/gemma_pytorch.git

Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 239 (delta 86), reused 53 (delta 53), pack-reused 121 (from 2)
Receiving objects: 100% (239/239), 2.18 MiB | 25.35 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [11]:
import sys

sys.path.append('gemma_pytorch')

In [12]:
from gemma.config import GemmaConfig, get_model_config
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
import contextlib
import os
import torch

モデルをセットアップ

In [13]:
# Set up model config.
model_config = get_model_config(CONFIG)
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT

# Instantiate the model and load the weights.
torch.set_default_dtype(model_config.get_dtype())
device = torch.device(MACHINE_TYPE)
model = GemmaForCausalLM(model_config)
model.load_weights(ckpt_path)
model = model.to(device).eval()

# 推論を実行する



In [14]:
# Generate with one request in chat mode

# Chat templates
USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn><eos>\n"
MODEL_CHAT_TEMPLATE = "<start_of_turn>model\n{prompt}<end_of_turn><eos>\n"

# Sample formatted prompt
prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt='What is a good place for travel in the US?'
    )
    + MODEL_CHAT_TEMPLATE.format(prompt='California.')
    + USER_CHAT_TEMPLATE.format(prompt='What can I do in California?')
    + '<start_of_turn>model\n'
)
print('Chat prompt:\n', prompt)

model.generate(
    USER_CHAT_TEMPLATE.format(prompt=prompt),
    device=device,
    output_len=128,
)

Chat prompt:
 <start_of_turn>user
What is a good place for travel in the US?<end_of_turn><eos>
<start_of_turn>model
California.<end_of_turn><eos>
<start_of_turn>user
What can I do in California?<end_of_turn><eos>
<start_of_turn>model



"California is a vast and diverse state with something for everyone!  To give you some great recommendations, tell me: \n\n* **What kind of activities do you enjoy?**  (e.g., hiking, beaches, history, art, nightlife, food)\n* **What part of California are you interested in?** (e.g., the coast, the mountains, the cities, etc.)\n* **What's your budget like?** (e.g., luxurious hotels, budget-friendly hostels, camping)\n* **Who are you travelling with?** (e.g., solo, as a"